In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# ======= CONFIGURATION =======
# Jira details
JIRA_URL = "https://icedq.atlassian.net/"
EMAIL = "tapshyu.g@toranainc.com"
API_TOKEN = "ATATT3xFfGF0OnMY5KM7c7RgR2fCXnBOdw8L4yGcVGfFjiRUBScbuHLnA8TpwIJLH51ZhPOLgYFTeAatqHeQTAr7EoiAw_ANZ3DlN08vZEofsNnND_7Bd7ej21LsDcaKAKPvOfj2J5wUukmB1xW2PK3TKk9EQcgwspEvBzz6cXB9QhkcNJHOlqk=C35E7E2E"

# Xray details
XRAY_URL = "https://us.xray.cloud.getxray.app/api/v2"
CLIENT_ID = "your-xray-client-id"
CLIENT_SECRET = "your-xray-client-secret"

# Project & Folder setup
PROJECT_KEY = "XRAY"
SOURCE_FOLDER_ID = "12345"               # Folder where original tests exist
DEST_FOLDER_NAME = "Cloned Tests"        # Name of new folder
PARENT_FOLDER_ID = None                  # Optional: put under another folder (set ID or None)
# =============================


# Jira Authentication
jira_auth = HTTPBasicAuth(EMAIL, API_TOKEN)
jira_headers = {"Accept": "application/json", "Content-Type": "application/json"}

# Xray Authentication
auth_response = requests.post(
    f"{XRAY_URL}/authenticate",
    json={"client_id": CLIENT_ID, "client_secret": CLIENT_SECRET},
    headers={"Content-Type": "application/json"}
)
if auth_response.status_code != 200:
    raise Exception("❌ Xray authentication failed: " + auth_response.text)
xray_token = auth_response.json()
xray_headers = {"Authorization": f"Bearer {xray_token}", "Content-Type": "application/json"}

# 1. Create new destination folder
new_folder_resp = requests.post(
    f"{XRAY_URL}/testrepository/{PROJECT_KEY}/folders",
    headers=xray_headers,
    json={"name": DEST_FOLDER_NAME, "parentFolderId": PARENT_FOLDER_ID}
)
if new_folder_resp.status_code not in [200, 201]:
    raise Exception("❌ Failed to create folder: " + new_folder_resp.text)
new_folder_id = new_folder_resp.json()["id"]
print(f"✅ Created new folder '{DEST_FOLDER_NAME}' (id={new_folder_id})")

# 2. Get tests inside source folder
tests_resp = requests.get(
    f"{XRAY_URL}/testrepository/{PROJECT_KEY}/folders/{SOURCE_FOLDER_ID}/tests",
    headers=xray_headers
)
if tests_resp.status_code != 200:
    raise Exception("❌ Failed to fetch source folder tests: " + tests_resp.text)

tests = tests_resp.json().get("tests", [])
print(f"📌 Found {len(tests)} tests in source folder.")

# 3. Loop and clone each test (including steps)
for test in tests:
    test_key = test["key"]

    # --- Fetch Jira issue details ---
    jira_issue = requests.get(
        f"{JIRA_URL}/rest/api/3/issue/{test_key}",
        headers=jira_headers,
        auth=jira_auth
    ).json()

    # --- Create new Jira test issue (clone) ---
    clone_data = {
        "fields": {
            "project": {"key": PROJECT_KEY},
            "summary": f"Clone of {jira_issue['fields']['summary']}",
            "issuetype": {"id": jira_issue['fields']['issuetype']['id']},
            "description": jira_issue['fields'].get("description", "")
        }
    }
    clone_resp = requests.post(
        f"{JIRA_URL}/rest/api/3/issue",
        headers=jira_headers,
        auth=jira_auth,
        data=json.dumps(clone_data)
    )
    if clone_resp.status_code not in [200, 201]:
        print(f"⚠️ Failed to clone {test_key}: {clone_resp.text}")
        continue

    new_test_key = clone_resp.json()["key"]
    print(f"✅ Created cloned test: {new_test_key}")

    # --- Fetch steps from source test ---
    steps_resp = requests.get(
        f"{XRAY_URL}/test/{test_key}/steps",
        headers=xray_headers
    )
    if steps_resp.status_code == 200:
        steps = steps_resp.json()
        for step in steps:
            step_data = {
                "action": step["action"],
                "data": step.get("data", ""),
                "result": step.get("result", "")
            }
            add_step_resp = requests.post(
                f"{XRAY_URL}/test/{new_test_key}/step",
                headers=xray_headers,
                json=step_data
            )
            if add_step_resp.status_code not in [200, 201]:
                print(f"⚠️ Failed to add step: {add_step_resp.text}")
        print(f"   ➕ Copied {len(steps)} steps to {new_test_key}")
    else:
        print(f"⚠️ Could not fetch steps for {test_key}")

    # --- Add cloned test to new folder ---
    add_resp = requests.post(
        f"{XRAY_URL}/testrepository/{PROJECT_KEY}/folders/{new_folder_id}/tests",
        headers=xray_headers,
        json={"add": [new_test_key]}
    )
    if add_resp.status_code not in [200, 201]:
        print(f"⚠️ Failed to add {new_test_key} to folder: {add_resp.text}")

print("🎉 All tests (with steps) cloned into new folder successfully!")


In [ ]:
import requests
from requests.auth import HTTPBasicAuth

JIRA_URL = "https://icedq.atlassian.net/"
EMAIL = "tapshyu.g@toranainc.com"
API_TOKEN = "your-api-token"

auth = HTTPBasicAuth(EMAIL, API_TOKEN)
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Example: Get issue
response = requests.get(
    f"{JIRA_URL}/rest/api/3/issue/PROJ-123",
    headers=headers,
    auth=auth
)

print(response.json())